In [1]:
# 곱셈계층 구현

class MulLayer:
  def __init__(self): # 인스턴스 변수 x,y 초기화
    self.x = None
    self.y = None

  def forward(self, x, y): # 순전파
    self.x = x # 입력한 x값으로 설정
    self.y = y # 입력한 y값으로 설정
    out = x * y # x와 y를 곱해서 출력
    return out

  def backward(self, dout): # 역전파
    dx = dout * self.y # 상류에서 미분한 값과 순전파 때 x였기 때문에 서로 바꿔서 y를 곱함
    dy = dout * self.x # 상류에서 미분한 값과 순전파 때 y였기 때문에 서로 바꿔서 x를 곱함
    return dx, dy

In [5]:
apple = 100
apple_num = 2
tax = 1.1

# 계층
mul_apple_layer = MulLayer() # 위에서 생성한 MulLayer()로 설정
mul_tax_layer = MulLayer() # 위에서 생성한 MulLayer()로 설정

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # 사과 가격을 사과의 원가와 개수를 MulLayer의 순전파 적용
price = mul_tax_layer.forward(apple_price, tax) # 순전파를 적용한 사과가격과 택스를 MulLayer의 순전파 적용

print(price)
print('-'*20)

# 역전파
dprice = 1 # 역전파의 처음 시작은 1
dapple_price, dtax = mul_tax_layer.backward(dprice) # dprice를 역전파를 통해서 하류의 dapple_price와 dtax를 적용
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # 상류에서 계산된 dapple_price를 역전파를 통해서 하류의 dapple, dapple_num를 적용

print(dapple, dapple_num, tax)

220.00000000000003
--------------------
2.2 110.00000000000001 1.1


In [6]:
# 덧셈계층 구현

class AddLayer:
  def __init__(self): # 덧셈 계층에는 초기화가 필요하지 않기 때문에 Pass
    pass
  
  def forward(self, x, y): # 순전파
    out = x + y # 입력된 x,y를 더해서 출력
    return out

  def backward(self, dout): # 역전파
    dx = dout * 1 # 상류에서 미분한 값으로 하류로 보냄
    dy = dout * 1 # 상류에서 미분한 값으로 하류로 보냄
    return dx, dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층
mul_apple_layer = MulLayer() # 위에서 생성한 MulLayer()로 설정
mul_orange_layer = MulLayer() # 위에서 생성한 MulLayer()로 설정
add_apple_orange_layer = AddLayer() # 위에서 생성한 AddLayer()로 설정
mul_tax_layer = MulLayer() # 위에서 생성한 MulLayer()로 설정

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)  # 사과 가격을 사과의 원가와 개수를 MulLayer의 순전파 적용
orange_price = mul_orange_layer.forward(orange, orange_num) # 오렌지 가격을 오렌지의 원가와 개수를 MulLayer의 순전파 적용
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # 사과 가격과 오렌지 가격을 AddLayer의 순전파 적용
price = mul_tax_layer.forward(all_price, tax)  # 순전파를 적용한 사과가격, 오렌지가격과 택스를 MulLayer의 순전파 적용

# 역전파
dprice = 1 # 역전파의 처음 시작은 1
dall_price, dtax = mul_tax_layer.backward(dprice) # dprice를 역전파를 통해서 하류의 dall_price와 dtax를 적용
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # dall_price를 역전파를 통해서 하류의 dapple_price와 dorange_price를 적용
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # dorange_price를 역전파를 통해서 하류의 dorange와 dorange_num를 적용
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # dapple_price를 역전파를 통해서 하류의 dapple와 dapple_num를 적용

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# ReLU함수식

#$h(x) = \begin{cases} x \ (x>0)\\ 0 \ (x\leq0) \end{cases}$



# ReLU 미분식

# $\frac{\delta{y}}{\delta{x}} = \begin{cases} 1 \ (x>0)\\ 0 \ (x\leq0) \end{cases}$

In [8]:
# ReLU함수 구현

class Relu:
  def __init__(self):
    self.mask = None # mask 인스턴스 변수 초기화

  def forward(self, x):
    self.mask = (x<=0) # x의 원소 값이 0 이하인 인덱스는 True, 그외는 False
    out = x.copy() # 출력은 x의 값을 복사
    out[self.mask] = 0 # 입력 값이 0 이하면 역전파 때의 값은 0으로 설정
    return out

  def backward(self, dout):
    dout[self.mask] = 0 # 순전파때 만들어둔 mask를 써서 원소가 True면 상류에서 전파된 dout을 0으로 설정
    dx = dout # x에 대한 미분으로 설정
    return dx

# Sigmoid 함수식

#$y = \frac{1}{1+e^{-x}}$


# Sigmoid 미분식

# $\frac{\delta{L}}{\delta{y}}y^2e^{-x} = \frac{e^{-x}}{{(1+e^{-x})}^2} = \frac{\delta{L}}{\delta{y}}y(1-y)$  

In [9]:
# Sigmoid함수 구현
import numpy as np

class Sigmoid:
  def __init__(self):
    self.out = None # 출력 초기화

  def forward(self, x):
    out = 1 / (1 + np.exp(-x)) # sigmoid 함수식 구현
    self.out = out
    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out # sigmoid 미분식 구현
    return dx

In [13]:
# 어파인 변환은 신경망의 순전파 때 수행하는 행렬의 곱을 의미
# Affine 구현

X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

y = np.dot(X,W) + B

print(y)

(2,)
(2, 3)
(3,)
[0.63067628 1.37393088 1.08974785]


In [14]:
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x): # 순전파는 행렬의 곱 연산
    self.x = x
    out = np.dot(x, self.W) + self.b
    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T) # 행렬의 미분은 역행렬로 이해
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0) # axis = 0은 0번째 축이므로 0번째 축의 총합
    return dx

In [15]:
def cross_entropy_error(y,t):
  if y.ndim == 1: # y의 차원이 1차원일때
    t = t.reshape(1, t.size) # t의 사이즈로 재형상
    y = y.reshape(1, y.size) # y의 사이즈로 재형상

  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size # 배치의 크기로 나눠 정규화하고 평균의 교차 엔트로피 오차를 계산


def softmax(a):
  c = np.max(a) # 오버플로 방지위해서 입력신호에서 가장 큰 수 설정
  exp_a = np.exp(a - c) # 오버플로 방지 
  sum_exp_a = np.sum(exp_a) # 지수함수 a의 합
  y = exp_a / sum_exp_a # 소프트맥스 함수 구현
  return y

In [17]:
class SoftmaxWithLoss:
  def __init__(self): # loss, y, t 초기화
    self.loss = None 
    self.y = None 
    self.t = None

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x) # 위에서 설정한 softmax 적용
    self.loss = cross_entropy_error(self.y, self.t) # loss 크로스엔트로피에러 적용
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0] # t의 차원수로 배치사이즈 설정
    dx = (self.y - self.t) / batch_size # 전파하는 값을 배치로 나눠서 전파
    return dx

In [19]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git
import sys, os
sys.path.append('/content/deep-learning-from-scratch')
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict() # 순서가 있는 딕셔너리
    self.layers['Affien1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    self.lastLayer = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    return x

  # x:입력, t:정답레이블

  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.nidm != 1 : t = np.argmax(t, axis=1)
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

  def gradient(self, x, t):
    # 순전파
    self.loss(x, t)

    # 역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    # 결과 저장
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].dW
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].dW
    return grads

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 826, done.
remote: Total 826 (delta 0), reused 0 (delta 0), pack-reused 826
Receiving objects: 100% (826/826), 52.21 MiB | 44.26 MiB/s, done.
Resolving deltas: 100% (477/477), done.


In [22]:
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 가중치의 차이의 절댓값을 구한후 그 절댓값들의 평균
for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
  print(key + ':' + str(diff))

W1:5.281659582144946e-10
b1:3.266011442445548e-09
W2:6.907127359673907e-09
b2:1.4092849031088762e-07


In [23]:
from dataset.mnist import load_mnist
from ch05.two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) # 입력 784개, 히든층 50개, 출력 10개

iters_num = 10000
train_size = x_train.shape[0] # x_train의 차원수만큼 크기 설정
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1) # train_size를 batch_size로 나눈 값의 최댓값으로 에포크 설정

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  grad = network.gradient(x_batch, t_batch) # 오차역전파로 기울기 설정

  for key in ('W1', 'b1', 'W2', 'b2'): # lr과 기울기로 params 갱신
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss) # 구한 loss값을 리스트화

  if i % iter_per_epoch == 0: # 에폭이 0이 될때까지
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc)

0.15003333333333332 0.1544
0.9044 0.907
0.92175 0.9248
0.9361166666666667 0.9367
0.9441 0.9438
0.9521333333333334 0.9511
0.9572166666666667 0.9546
0.9605166666666667 0.9555
0.9632833333333334 0.9609
0.9643666666666667 0.9608
0.9675 0.963
0.9701833333333333 0.9652
0.9717166666666667 0.9663
0.9726333333333333 0.9671
0.9745166666666667 0.9668
0.9753166666666667 0.9691
0.9769666666666666 0.9702
